In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath("."), 'src'))

In [3]:
from ato_tools import scoring

In [9]:
import importlib
importlib.reload(testing)

<module 'ato_tools.testing' from 'C:\\wfrc\\ato\\src\\ato_tools\\testing.py'>

In [46]:
if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(scoring)

In [4]:
scoring.score("baseline.gdb", out_table="auto_baseline", mode = "Driving", test = True)

Regional Jobs per HH Ratio: 2.211549546877222
Scores written to baseline.gdb\auto_baseline_summary
Network ATO: 14820078.0


In [2]:
from scoring import score

ModuleNotFoundError: No module named 'scoring'

In [8]:
import arcpy
import os
import errno
from arcgis.features import SpatialDataFrame
import pandas as pd
import math

arcpy.CheckOutExtension("network")

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

In [29]:

nd_gdb = "baseline.gdb"
out_table="auto_baseline"
mode = "Driving"
test = False
nd = r"NetworkDataset\NetworkDataset_ND"

base_path = os.path.abspath(".")

arcpy.env.workspace = os.path.join(base_path, "ato.gdb")

# location of the file geodatabase with the WFRC TAZ shapes and TAZ centroids    
ato_gdb = os.path.join(base_path, r"shp\taz_wfrc.gdb")

if test:
    centroids = os.path.join(ato_gdb, "taz_centroids_sample")
else:
    centroids = os.path.join(ato_gdb, "taz_centroids_snapped")

nd_layer_name = "wfrc_mm"

nd_path = os.path.join(nd_gdb, nd)

if arcpy.Exists(nd_path):
    arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)
else:
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), nd_path)

odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)

odcm.travelMode = mode

# do not consider travel times beyond 60 minutes
odcm.defaultImpedanceCutoff = 60 

odcm.lineShapeType = arcpy.nax.LineShapeType.NoLine

# Load inputs using field mappings, including the pre-calculated location fields
field_mappings_origins = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, True)

# Load origins, mapping the "CO_TAZID" field to the "Name" property of the Origins class
field_mappings_origins["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, centroids, field_mappings_origins)

# Load destinations, mapping the "CO_TAZID" field to the "Name" property of the Origins class 
field_mappings_destinations = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, True)
field_mappings_destinations["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, centroids, field_mappings_destinations)


# Solve the OD skim matrix
result = odcm.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, r"memory\output_lines")
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

od = pd.DataFrame.spatial.from_featureclass(r"memory\output_lines")

if od['Total_Time'].mean() < 1:
    print("Network validation: FAIL")
    print("Travel Times: ", od['Total_Time'].head())
    raise ValueError('Network Travel Times are Zero - Invalid Network')

od['DestinationName'] = od['DestinationName'].astype(int)

taz = pd.DataFrame.spatial.from_featureclass(os.path.join(ato_gdb, "ATO"))

taz.drop(columns=taz.columns.difference(['CO_TAZID', 'HH_19', 'JOB_19']), inplace=True) #, 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

# rename TAZ fields to make this easier to accomodate updates in future years
taz.rename(columns={'HH_19': 'HH', 'JOB_19': 'JOB'}, inplace=True)

region_job_per_hh = taz['JOB'].sum() / taz['HH'].sum()
print("Regional Jobs per HH Ratio: {}".format(region_job_per_hh))

df = pd.merge(od, taz, left_on="DestinationName", right_on="CO_TAZID")

df.rename(columns={"OriginName": "Origin_TAZID",
                   "DestinationName": "Destination_TAZID"},
          inplace=True)

# Weight outputs
df['survey_weight'] = df['Total_Time'].apply(lambda x: survey_weight(x)).round(3)

df['accessible_jobs'] = round(df['survey_weight'] * df['JOB'])
df['accessible_hh'] = round(df['survey_weight'] * df['HH'])



Regional Jobs per HH Ratio: 1.808754179036092


In [30]:
df = pd.merge(od, taz, left_on="DestinationName", right_on="CO_TAZID")

df.rename(columns={"OriginName": "Origin_TAZID",
                   "DestinationName": "Destination_TAZID"},
          inplace=True)

# Weight outputs
df['survey_weight'] = df['Total_Time'].apply(lambda x: survey_weight(x)).round(3)

df['accessible_jobs'] = round(df['survey_weight'] * df['JOB'])
df['accessible_hh'] = round(df['survey_weight'] * df['HH'])


In [31]:

# keep only relevant columns
df.drop(columns=df.columns.difference(['Origin_TAZID', 'Destination_TAZID', 'Total_Time', 
                                       'accessible_jobs', 'accessible_hh']), 
        inplace=True)

df.spatial.to_table(os.path.join(nd_gdb, out_table))

# save table to input GDB
#arcpy.conversion.TableToTable(r'tmp\scores.csv', nd_gdb, out_name)

df_summary = df.groupby('Origin_TAZID').agg(
    accessible_jobs=pd.NamedAgg(column='accessible_jobs', aggfunc=sum),
    accessible_hh=pd.NamedAgg(column='accessible_hh', aggfunc=sum)
)

In [32]:
df_summary.head()

,accessible_jobs,accessible_hh
Origin_TAZID,,
110424,85717.0,61552.0
110425,119959.0,81117.0
110426,132554.0,87426.0
110427,127956.0,83118.0
110428,124408.0,84669.0


In [33]:
df_summary['CO_TAZID'] = df_summary.index.astype(int)

In [34]:
# need to preserve left index
taz_ato = pd.merge(df_summary, taz, on="CO_TAZID")
taz_ato.head()

,accessible_jobs,accessible_hh,CO_TAZID,HH,JOB
0,85717.0,61552.0,110424,70.0,2.0
1,119959.0,81117.0,110425,61.7,0.0
2,132554.0,87426.0,110426,626.3,117.5
3,127956.0,83118.0,110427,1142.7,65.3
4,124408.0,84669.0,110428,121.7,1.2


In [35]:
# Apply WFRC TAZ-based ATO formula
taz_ato['ato'] = round((taz_ato['accessible_hh'] * taz_ato['JOB'] + taz_ato['accessible_jobs'] * taz_ato['HH'] * region_job_per_hh) / 
                  (taz_ato['HH'] * region_job_per_hh + taz_ato['JOB']))

#taz_summary.to_csv(r'tmp\scores_summary.csv')
taz_ato.spatial.to_table(os.path.join(nd_gdb, out_table + "_summary"))

# save table to input GDB
#arcpy.conversion.TableToTable(r'tmp\scores_summary.csv', nd_gdb, "scores_summary")

print("Scores written to {}".format(os.path.join(nd_gdb, out_table + "_summary")))

print("Network ATO: {}".format(taz_ato['ato'].sum()))

Scores written to baseline.gdb\auto_baseline_summary
Network ATO: 624751677.0
